In [ ]:
from source.class_yolo_trainer import YOLOTrainer

yolo_trainer = YOLOTrainer
yolo_trainer = YOLOTrainer()

In [ ]:
### vou deixar anotado porque não sei se fiz gambiarra, 
""" no atributo image_folder para tarefas de classificação eu estou passando somente a pasta do dataset, sem indicar a pasta de imagens
"""
yolo_trainer.image_folder = r"emissoes_dataset_YOLO\images"
yolo_trainer.annotations_folder = r"emissoes_dataset_YOLO\labels"
yolo_trainer.yolo_Classes = ["emissao"]
yolo_trainer.dataset_path = "emissoes_YOLO"
yolo_trainer.task = "detect"
yolo_trainer.aug = True
yolo_trainer.n_aug = 10
yolo_trainer.odd = 0.12
yolo_trainer.test_percentual_divisor = 20

yolo_trainer.slicing()

In [1]:
from source.modules.opt_yolo import objective
from ultralytics import YOLO
import optuna

model = YOLO(r"yolov10m.pt")

# model = r"YOLO_tools\yolo11m.pt"

images_dir = r'D:\Judson_projetos\Yolo_trainer\YOLO_tools\datasets\emissoes_YOLO\images'
labels_dir = r'D:\Judson_projetos\Yolo_trainer\YOLO_tools\datasets\emissoes_YOLO\labels'
data_yaml = r'D:\Judson_projetos\Yolo_trainer\YOLO_tools\datasets\emissoes_YOLO\dataset.yaml'

storage_url = "sqlite:///emissoes-fugitivas.db"
study = optuna.create_study(direction="maximize", storage=storage_url, study_name="opt-emissoes", load_if_exists=True)      ### study for F1 metric, thats why maximize
study.optimize(lambda trial: objective(trial, images_dir, labels_dir, model, data_yaml), n_trials=50)

d:\Judson_projetos\Yolo_trainer\YOLO_tools\yolo-trainer-venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-01-06 12:50:49,616] A new study created in RDB with name: opt-emissoes
d:\Judson_projetos\Yolo_trainer\YOLO_tools\yolo-trainer-venv\Lib\site-packages\optuna\trial\_trial.py:678: RuntimeWarning: Inconsistent parameter values for distribution with name "epochs"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more than once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'log': False, 'step': 1, 'low': 5, 'high': 30}
  warnings.warn(


Ultralytics 8.3.58  Python-3.12.3 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)
engine\trainer: task=detect, mode=train, model=yolov10m.pt, data=coco8.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train25, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxe

train: Scanning D:\Judson_projetos\datasets\coco8\labels\train.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning D:\Judson_projetos\datasets\coco8\labels\val.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]


...


100 epochs completed in 0.064 hours.
Optimizer stripped from d:\Judson_projetos\Yolo_trainer\runs\detect\train25\weights\last.pt, 33.7MB
Optimizer stripped from d:\Judson_projetos\Yolo_trainer\runs\detect\train25\weights\best.pt, 33.7MB

Validating d:\Judson_projetos\Yolo_trainer\runs\detect\train25\weights\best.pt...
Ultralytics 8.3.58  Python-3.12.3 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)
YOLOv10m summary (fused): 369 layers, 16,543,024 parameters, 0 gradients, 63.9 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.06it/s]


                   all          4         17      0.829      0.843      0.916      0.764
                person          3         10          1       0.56      0.852      0.502
                   dog          1          1      0.866          1      0.995      0.995
                 horse          1          2      0.811          1      0.995      0.802
              elephant          1          2      0.878        0.5      0.662      0.398
              umbrella          1          1      0.694          1      0.995      0.995
          potted plant          1          1      0.727          1      0.995      0.895
Speed: 0.7ms preprocess, 41.7ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to d:\Judson_projetos\Yolo_trainer\runs\detect\train25


[W 2025-01-06 12:55:00,543] Trial 0 failed with parameters: {'epochs': 24, 'learning_rate': 0.0071745005165230885, 'batch_size': 52, 'weight_decay': 0.00562674622040046, 'step_size': 589, 'c_gamma': 0.047362145104801766, 'size': 640} because of the following error: RuntimeError('stack expects each tensor to be equal size, but got [1, 5] at entry 0 and [2, 5] at entry 10').
Traceback (most recent call last):
  File "d:\Judson_projetos\Yolo_trainer\YOLO_tools\yolo-trainer-venv\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\AUTOU4\AppData\Local\Temp\ipykernel_12752\1324809820.py", line 15, in <lambda>
    study.optimize(lambda trial: objective(trial, images_dir, labels_dir, model, data_yaml), n_trials=50)
                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Judson_projetos\Yolo_trainer\YOLO_tools\source\modules\opt_yolo.py", line 81,

RuntimeError: stack expects each tensor to be equal size, but got [1, 5] at entry 0 and [2, 5] at entry 10

In [ ]:
import torch
print(torch.cuda.is_available())

In [ ]:
import torch
import torchvision
print(f"PyTorch version: {torch.__version__}")
print(f"TorchVision version: {torchvision.__version__}")
print(torchvision.ops.nms.__module__)



In [ ]:
from ultralytics import YOLO
import yaml

#### YOLO é gambiarra e eu posso provar:
""" Para treinamentos de classificação com YOLO, você deve indicar o dir com o dataset
que deve estar especificado dentro de uma pasta chamada 'datasets'. No entando, para detecção
o YOLO é diferente. Você precisa indicar o caminho do arquivo 'dataset.yaml' para que ele 
possa encontrar o dataset e realizar o treinamento. É a mesma função, de uma mesma lib,
mas os caras fizeram de forma que o mesmo argumento recebe duas entradas completamente 
diferentes a depender do treinamento que você vai fazer.
"""

# Carregar configurações de um arquivo YAML
with open('../hyper_yolo.yaml', 'r') as file:
    config = yaml.safe_load(file)

train_config = config['train']
aug_config = config['train']['augmentation']

model = YOLO("runs\\detect\\train52\\weights\\best.pt")

results = model.train(
    data = "datasets/emissoes_YOLO/dataset.yaml",
    device = "cuda",

    ### training configs. Checkout hyper_yolo.yaml for details
    imgsz = train_config['imgsz'],
    batch = train_config['batch'],
    weight_decay = train_config['weight_decay'],

    warmup_epochs = train_config['warmup_epochs'],
    warmup_momentum = train_config['warmup_momentum'],
    warmup_bias_lr = train_config['warmup_bias_lr'],

    epochs = train_config['epochs'],
    momentum = train_config['momentum'],
    lr0 = train_config['lr0'],
    lrf = train_config['lrf'],
    optimizer = train_config['optimizer'],

    ### if you r willing to use yolo aug, uncomment section bellow
    ### not recomended if ur data is already aug though
    ### augment args
    # hsv_h = aug_config['hsv_h'],
    # hsv_s = aug_config['hsv_s'],
    # hsv_v = aug_config['hsv_v'],
    # degrees = aug_config['degrees'],
    # translate = aug_config['translate'],
    # scale = aug_config['scale'],
    # shear = aug_config['shear'],
    # perspective = aug_config['perspective'],
    # flipud = aug_config['flipud'],
    # fliplr = aug_config['fliplr'],
    # mosaic = aug_config['mosaic'],
    # mixup = aug_config['mixup'],
    # copy_paste = aug_config['copy_paste'],
    # auto_augment = aug_config['auto_augment'],
)

In [ ]:
from ultralytics import YOLO

model = YOLO("runs\\detect\\train52\\weights\\best.pt")

model.info()

In [ ]:
from ultralytics import YOLO

train = 'train52'
model = YOLO(f"runs/detect/{train}/weights/best.pt")
model.predict("../RUIM 23-06-22 (1).mp4", save=True, conf=0.4, device="cuda", save_txt=False, save_conf=True, save_crop=False)

In [ ]:
from utils.check_file import check_file

check_file("datasets\\emissoes_YOLO\\dataset.yaml")

In [ ]:
from utils.torch_is_available import torch_is_available

torch_is_available()

In [ ]:
from utils.cvat_dataset import cvat_dataset

cvat_dataset("../emissoes_dataset_CVAT", "emissoes_dataset_YOLO")

In [ ]:
from utils.remover_labels_vazios import processar_dataset

processar_dataset("emissoes_dataset_YOLO")